# CFM 301 Data Assignment 2 
## Q3
### Jeongseop Yi (Patrick), j22yi

In [1]:
# get data from downloaded files
import pandas as pd
from pandasql import sqldf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# read q1 data 
q1_data = pd.read_sas('../q1.sas7bdat', encoding='latin-1')
q1_data.columns = map(str.upper, q1_data.columns)
q1_data['DATE'] = pd.to_datetime(q1_data['DATE'], format='%Y%m%d')
q1_data['MONTHID'] = (q1_data['DATE'].dt.year - 2000) * 12 + q1_data['DATE'].dt.month
display(q1_data.count())

# read q2 data
q2_data = pd.read_sas('../q2.sas7bdat', encoding='latin-1')
q2_data.columns = map(str.upper, q2_data.columns)
q2_data['DATE'] = pd.to_datetime(q2_data['DATADATE'], format='%Y%m%d')
q2_data['MONTHID2'] = (q2_data['DATE'].dt.year - 2000) * 12 + q2_data['DATE'].dt.month

# read permno and gvkey data
permno_gvkey = pd.read_excel('../downloads/nasdaq100_pop.xlsx', sheet_name='all', dtype={'GVKEY': str})
permno_gvkey.columns = map(str.upper, permno_gvkey.columns)

PERMNO     22819
DATE       22819
CUSIP      22819
COMNAM     22773
TICKER     22773
PRC        22769
RET        22721
RETX       22721
SHROUT     22773
VWRETD     22819
SPRTRN     22819
MONTHID    22819
dtype: int64

In [3]:
# There exist negative prices, so we take the absolute value
# It is not an error but a notation from CRSP
# When no closing price is available, the price is flagged as negative
q1_data['PRC'] = abs(q1_data['PRC'])
q1_data = pd.merge(q1_data, permno_gvkey[['PERMNO', 'GVKEY']], on='PERMNO', how='left')

In [4]:
merged_data = sqldf("SELECT a.*, b.* \
                     FROM q1_data as a \
                     LEFT JOIN q2_data as b \
                     ON a.GVKEY = b.GVKEY and a.MONTHID >= b.MONTHID2 + 4 and a.MONTHID <= b.MONTHID2 + 6")

# keep data from 2000 to 2021
merged_data = merged_data[merged_data['MONTHID'] >= 1]

In [5]:
# save data to xlsx
with pd.ExcelWriter("../DA2_data.xlsx", mode='a', engine="openpyxl", if_sheet_exists='replace') as writer:
    merged_data.to_excel(writer, sheet_name="q3", index=False)

KeyboardInterrupt: 